In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pyccl as ccl

In [ ]:
cosmo = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda = ccl.Cosmology(
    Omega_c=0.25, Omega_b=0.05,
    w0 = 0., wa = 0,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

cosmo_no_lambda_v2 = ccl.Cosmology(
    Omega_c=0.94, Omega_b=0.05,
    h=0.7, n_s=0.95, sigma8=0.8,
    transfer_function='bbks'
)

In [ ]:
def z_to_a(z: np.ndarray) -> np.ndarray:
    return 1/(1+z)

def a_to_z(a: np.ndarray) -> np.ndarray:
    return 1/(1-a)

In [ ]:
z_grid = np.linspace(0.1, 30, 291)
a_grid = z_to_a(z_grid)

In [ ]:
_ = plt.plot(z_grid, cosmo.comoving_radial_distance(a_grid), label='LCDM')
_ = plt.plot(z_grid, cosmo_no_lambda.comoving_radial_distance(a_grid), label='wo = 0')
_ = plt.plot(z_grid, cosmo_no_lambda_v2.comoving_radial_distance(a_grid), label='omega_b = 0.94')
_ = plt.legend()


In [ ]:
mesh = np.meshgrid(z_grid, z_grid)

In [ ]:
def generalized_sin(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return np.sin(scaleK*x)/scaleK
    elif K > 0:
        return np.sinh(scaleK*x)/scaleK
    return x

def generalized_cos(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return np.cos(scaleK*x)/scaleK
    elif K > 0:
        return np.cosh(scaleK*x)/scaleK
    return np.ones(len(x))

def generalized_cot(x: np.ndarray, K: float=0) -> np.ndarray:
    scaleK = np.sqrt(np.fabs(K))
    if K < 0:
        return scaleK/np.tan(scaleK*x)
    elif K > 0:
        return scaleK/np.tanh(scaleK*x)
    return np.ones(len(x))



In [ ]:
def ratio(cosmo, z_d: np.ndarray, z_l: np.ndarray) -> np.ndarray:
    a_d = z_to_a(z_d)
    a_l = z_to_a(z_l)
    a_s = a_d * a_l
    chi_d = cosmo.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
    chi_s = cosmo.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
    chi_l = chi_s - chi_d
    return (1 + (chi_d/chi_l))/a_s

In [ ]:
z_d = mesh[0]
z_l = mesh[1]
a_d = z_to_a(z_d)
a_l = z_to_a(z_l)
a_s = a_d * a_l
chi_d = cosmo_no_lambda.comoving_radial_distance(a_d.ravel()).reshape(a_d.shape)
chi_s = cosmo_no_lambda.comoving_radial_distance(a_s.ravel()).reshape(a_s.shape)
chi_l = chi_s - chi_d
rat_chi = chi_d/chi_l

In [ ]:
rat_chi

In [ ]:
r = ratio(cosmo, mesh[0], mesh[1])
r1 = ratio(cosmo_no_lambda, mesh[0], mesh[1])

In [ ]:
_ = plt.imshow(r/r1, origin='lower', norm='log', extent=(z_grid[0], z_grid[-1], z_grid[0], z_grid[-1]))
_ = plt.colorbar(label=r"$D_{\Delta T}/D_{D}$")
_ = plt.xlabel(r'$z_{\rm s}$')
_ = plt.ylabel(r'$z_{\rm d}$')